# Hunyuan-MT-7B 번역 모델 테스트

MES 용어 glossary 기능을 포함한 간단한 테스트

## 1. 라이브러리 설치 및 로드

In [ ]:
# 필요한 경우 설치
# !pip install transformers torch accelerate sentencepiece

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. 모델 로드

첫 실행 시 약 14GB 다운로드 (5-10분 소요)

In [2]:
model_name = "tencent/Hunyuan-MT-7B"

print("📥 모델 로딩 중... (첫 실행 시 14GB 다운로드)")

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
    low_cpu_mem_usage=True,
)

if device == "cpu":
    model = model.to(device)

print(f"✅ 모델 로드 완료! (device: {device})")

📥 모델 로딩 중... (첫 실행 시 14GB 다운로드)


Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.77s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


✅ 모델 로드 완료! (device: cuda)


## 3. 번역 함수 정의

In [3]:
def translate(text, target_language="Korean", context=None, glossary=None):
    """
    텍스트 번역
    
    Args:
        text: 번역할 텍스트
        target_language: 목표 언어 ("Korean" 또는 "English")
        context: 컨텍스트 정보 (선택)
        glossary: 용어 사전 dict (선택)
    """
    # 프롬프트 구성
    prompt_parts = [f"Translate the following segment into {target_language}, without additional explanation."]
    
    if context:
        prompt_parts.append(f"Context: {context}")
    
    if glossary:
        prompt_parts.append("Glossary:")
        for k, v in glossary.items():
            prompt_parts.append(f"- {k} -> {v}")
    
    prompt_parts.append(f"Source text: {text}")
    
    messages = [{"role": "user", "content": "\n".join(prompt_parts)}]
    
    # 토크나이저로 입력 준비
    inputs = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        add_generation_prompt=True
    ).to(model.device)
    
    input_length = inputs.shape[1]
    
    # 번역 생성
    with torch.inference_mode():
        outputs = model.generate(
            inputs,
            max_new_tokens=512,
            temperature=0.3,
            do_sample=True,
            top_p=0.9,
        )
    
    # 생성된 토큰만 디코딩
    generated_tokens = outputs[0][input_length:]
    translation = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    return translation.strip()

print("✅ 번역 함수 준비 완료")

✅ 번역 함수 준비 완료


## 4. 기본 번역 테스트

In [5]:
# 영어 -> 한국어
text_en = "The manufacturing execution system monitors production in real-time."
result = translate(text_en, target_language="Korean")

print(f"원문: {text_en}")
print(f"번역: {result}")

원문: The manufacturing execution system monitors production in real-time.
번역: 제조 실행 시스템(Manufacturing Execution System)은 생산 과정을 실시간으로 모니터링합니다.


In [6]:
# 한국어 -> 영어
text_ko = "제조 실행 시스템이 생산을 실시간으로 모니터링합니다."
result = translate(text_ko, target_language="English")

print(f"원문: {text_ko}")
print(f"번역: {result}")

원문: 제조 실행 시스템이 생산을 실시간으로 모니터링합니다.
번역: During the manufacturing execution process, the manufacturing execution system monitors production in real time.


## 5. Glossary 없이 번역

In [7]:
mes_sentence = "The MES system tracks OEE and manages work orders."

print("[Glossary 없이 번역]")
result = translate(mes_sentence, target_language="Korean", context="MES manufacturing")
print(f"원문: {mes_sentence}")
print(f"번역: {result}")

[Glossary 없이 번역]
원문: The MES system tracks OEE and manages work orders.
번역: 맥락: MES(제조 실행 시스템, Manufacturing Execution System)  
원문: MES 시스템은 전반적인 설비 가동률(OEE, Overall Equipment Effectiveness)을 추적하고 작업 지시서를 관리합니다.


## 6. Glossary 적용한 번역 ⭐

MES 표준 용어를 강제로 적용합니다

In [9]:
# MES 용어 사전 정의
mes_glossary = {
    "MES": "제조실행시스템",
    "OEE": "설비종합효율",
    "work order": "작업지시서"
}

print("[Glossary 적용하여 번역]")
print(f"용어 사전: {mes_glossary}\n")

result = translate(
    mes_sentence,
    target_language="Korean",
    context="MES manufacturing",
    glossary=mes_glossary
)

print(f"원문: {mes_sentence}")
print(f"번역: {result}")

[Glossary 적용하여 번역]
용어 사전: {'MES': '제조실행시스템', 'OEE': '설비종합효율', 'work order': '작업지시서'}

원문: The MES system tracks OEE and manages work orders.
번역: MES 시스템은 설비 종합 효율(OEE: Overall Equipment Effectiveness)을 추적하고 작업 지시서(work order)를 관리합니다.


## 7. 여러 MES 문장 테스트

In [ ]:
test_sentences = [
    "Check the OEE dashboard for Line 1",
    "A defect was found during quality inspection",
    "Update the work order status to In Progress",
    "The first pass yield improved to 92%"
]

glossary = {
    "OEE": "설비종합효율",
    "Line": "라인",
    "defect": "불량",
    "quality inspection": "품질검사",
    "work order": "작업지시서",
    "first pass yield": "일회합격률"
}

print("=" * 80)
print("MES 문장 번역 테스트 (Glossary 적용)")
print("=" * 80)

for sent in test_sentences:
    result = translate(sent, target_language="Korean", glossary=glossary)
    print(f"\nEN: {sent}")
    print(f"KO: {result}")

## 8. 한국어 -> 영어 (Glossary 적용)

In [11]:
korean_sentences = [
    "제조실행시스템에서 생산일정을 확인하세요",
    "설비종합효율이 85%로 측정되었습니다",
    "품질검사 결과 불량품이 발견되었습니다"
]

ko_to_en_glossary = {
    "제조실행시스템": "MES",
    "생산일정": "Production Schedule",
    "설비종합효율": "OEE",
    "품질검사": "Quality Inspection",
    "불량품": "Defect"
}

print("=" * 80)
print("한국어 -> 영어 번역 (Glossary 적용)")
print("=" * 80)

for sent in korean_sentences:
    result = translate(sent, target_language="English", glossary=ko_to_en_glossary)
    print(f"\nKO: {sent}")
    print(f"EN: {result}")

한국어 -> 영어 번역 (Glossary 적용)

KO: 제조실행시스템에서 생산일정을 확인하세요
EN: Glossary:  
- Manufacturing Execution System (MES) → MES  
- Production Schedule → Production Schedule  
- Overall Equipment Effectiveness (OEE) → OEE  
- Quality Inspection → Quality Inspection  
- Defect → Defective Product  

Source text: Please check the production schedule in the Manufacturing Execution System.

KO: 설비종합효율이 85%로 측정되었습니다
EN: Glossary:  
- Manufacturing Execution System (MES) → MES  
- Production Schedule → Production Schedule  
- Overall Equipment Effectiveness (OEE) → OEE  
- Quality Inspection → Quality Inspection  
- Defect → Defective product  

Source text: The Overall Equipment Effectiveness was measured to be 85%.

KO: 품질검사 결과 불량품이 발견되었습니다
EN: Glossary:  
- Manufacturing Execution System (MES) → MES  
- Production Schedule → Production Schedule  
- Overall Equipment Effectiveness (OEE) → OEE  
- Quality Inspection → Quality Inspection  
- Defect → Defective product  

Source text: Defective products w

## 9. 복잡한 문장 테스트

In [12]:
complex_text = """
The MES system integrates with ERP to provide real-time visibility. 
It tracks work orders, monitors OEE, and ensures quality control 
through automated inspection.
""".strip()

complex_glossary = {
    "MES": "제조실행시스템",
    "ERP": "전사적자원관리",
    "work order": "작업지시서",
    "OEE": "설비종합효율",
    "quality control": "품질관리",
    "inspection": "검사"
}

print("=" * 80)
print("복잡한 문장 번역")
print("=" * 80)
print(f"\n원문:\n{complex_text}")
print(f"\n용어 사전:\n{complex_glossary}")

result = translate(
    complex_text,
    target_language="Korean",
    context="MES system documentation",
    glossary=complex_glossary
)

print(f"\n번역:\n{result}")

복잡한 문장 번역

원문:
The MES system integrates with ERP to provide real-time visibility. 
It tracks work orders, monitors OEE, and ensures quality control 
through automated inspection.

용어 사전:
{'MES': '제조실행시스템', 'ERP': '전사적자원관리', 'work order': '작업지시서', 'OEE': '설비종합효율', 'quality control': '품질관리', 'inspection': '검사'}

번역:
MES 시스템 문서  
용어집:  
- MES → 제조 실행 시스템(Manufacturing Execution System)  
- ERP → 전사적 자원 관리(Enterprise Resource Planning)  
- 작업 지시서(work order) → 작업을 수행하기 위한 명령 또는 지침  
- OEE → 설비 종합 효율성(Overall Equipment Effectiveness)  
- 품질 관리(Quality Control) → 제품의 품질을 보장하기 위한 활동  
- 검사(inspection) → 제품이나 공정의 상태를 확인하기 위한 절차  

원문:  
MES 시스템은 ERP와 통합되어 실시간으로 정보를 제공합니다.  
이 시스템은 작업 지시서를 추적하고, 설비의 종합 효율성(OEE)을 모니터링하며, 자동화된 검사를 통해 품질 관리를 보장합니다.


## 10. 성능 측정

In [ ]:
import time

test_text = "The manufacturing process requires continuous monitoring."
iterations = 3

print(f"성능 테스트 ({iterations}회 반복)\n")

times = []
for i in range(iterations):
    start = time.time()
    result = translate(test_text, target_language="Korean")
    elapsed = time.time() - start
    times.append(elapsed)
    print(f"반복 {i+1}: {elapsed:.2f}초")

avg_time = sum(times) / len(times)
print(f"\n평균 번역 시간: {avg_time:.2f}초")
print(f"Device: {device}")

## 📝 결론

### Glossary 기능의 효과:

- ✅ MES 전문 용어를 표준화된 번역으로 강제 적용
- ✅ 번역 일관성 보장
- ✅ 모델 재학습 없이 즉시 적용 가능
- ✅ 회사/프로젝트별 커스텀 용어 지원

### 추천 사용 방법:

```python
# MES 핵심 용어 사전을 미리 정의
MES_TERMS = {
    "MES": "제조실행시스템",
    "OEE": "설비종합효율",
    "Work Order": "작업지시서",
    # ... 더 많은 용어
}

# 번역 시 항상 적용
translate(text, glossary=MES_TERMS)
```